In [0]:
%run "../commons/configuration"

In [0]:
%run "../commons/common_functions"

In [0]:
dbutils.widgets.text("env_name","DEV")
p_env_name = dbutils.widgets.get("env_name")

In [0]:
dbutils.widgets.text("file_date","2021-03-21")
p_file_date = dbutils.widgets.get("file_date")

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import to_timestamp, concat, lit, col

constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

constructors_df = spark.read.schema(constructors_schema).json(f"{raw_mount_path}/{p_file_date}/constructors.json")\
   .drop("url")\
   .withColumnRenamed("constructorId", "constructor_id")\
   .withColumnRenamed("contructorRef", "constructor_ref")\
   .withColumn("data_source",lit(p_env_name))\
   .withColumn("file_date",lit(p_file_date))

In [0]:
constructors_df_v2  = add_ingestion_date(constructors_df)

In [0]:
#constructors_df_v2.write.parquet(f"{processed_mount_path}/constructors", mode="overwrite")
constructors_df_v2.write.mode("overwrite").format("parquet").saveAsTable("processed.constructors")

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT * FROM processed.constructors